In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StandardScaler

In [ ]:
spark = SparkSession.builder.appName('RespiratoryDisease').getOrCreate()

#**Part 1 - Data Preprocessing**

In [ ]:
df = spark.read.csv('/FileStore/tables/metadata.csv',inferSchema=True,header=True)

df.show(5)

+---------+------+---+---+--------+--------+----+--------+--------------------+----+
patientid|offset|sex|age| finding|survival|view|modality| location| _c9|
+---------+------+---+---+--------+--------+----+--------+--------------------+----+
 2| 0| M| 65|COVID-19| Y| PA| X-ray|Cho Ray Hospital,...|null|
 2| 3| M| 65|COVID-19| Y| PA| X-ray|Cho Ray Hospital,...|null|
 2| 5| M| 65|COVID-19| Y| PA| X-ray|Cho Ray Hospital,...|null|
 2| 6| M| 65|COVID-19| Y| PA| X-ray|Cho Ray Hospital,...|null|
 4| 0| F| 52|COVID-19| null| PA| X-ray|Changhua Christia...|null|
+---------+------+---+---+--------+--------+----+--------+--------------------+----+
only showing top 5 rows

**1.1 Cleaning Null Values**

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show() #the survival label is mostly null, so lets remove it anyway since this is unsupervised. Also remove _c9 column which is entirely null

+---------+------+---+---+-------+--------+----+--------+--------+---+
patientid|offset|sex|age|finding|survival|view|modality|location|_c9|
+---------+------+---+---+-------+--------+----+--------+--------+---+
 0| 96| 43| 54| 0| 256| 0| 0| 118|368|
+---------+------+---+---+-------+--------+----+--------+--------+---+

In [ ]:
df = df.drop('survival','_c9')
df.columns #survival and _c9 columns are now dropped

Out[6]: ['patientid',
 'offset',
 'sex',
 'age',
 'finding',
 'view',
 'modality',
 'location']

In [ ]:
df = df.na.drop(how = 'any',) #so dropping rows where all the row values are null
df.describe().show()

+-------+----------------+-----------------+----+------------------+-------------+----+--------+-------------+
summary| patientid| offset| sex| age| finding|view|modality| location|
+-------+----------------+-----------------+----+------------------+-------------+----+--------+-------------+
 count| 185| 185| 185| 185| 185| 185| 185| 185|
 mean| 91.2|6.837837837837838|null| 55.68108108108108| null|null| null| null|
 stddev|69.5304750201071|7.157822998807399|null|15.106898759482696| null|null| null| null|
 min| 2| -5| F| 23| ARDS| AP| CT| Australia|
 max| 205| 35| M| 88|Streptococcus| PA| X-ray|Zigong, China|
+-------+----------------+-----------------+----+------------------+-------------+----+--------+-------------+

**1.2 Encoding Categorical Data**


In [ ]:
encoder_gen = StringIndexer(inputCol="sex", outputCol="sex_cat")
encoded = encoder_gen.fit(df).transform(df)

encoder_find = StringIndexer(inputCol="finding", outputCol="finding_cat")
encoded = encoder_find.fit(encoded).transform(encoded)

encoder_view = StringIndexer(inputCol="view", outputCol="view_cat")
encoded = encoder_view.fit(encoded).transform(encoded)

encoder_mod = StringIndexer(inputCol="modality", outputCol="modality_cat")
encoded = encoder_mod.fit(encoded).transform(encoded)

encoder_loc = StringIndexer(inputCol="location", outputCol="location_cat")
encoded = encoder_loc.fit(encoded).transform(encoded)

encoded.show(5) #notice new column that encodes the ocean prox data

+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+
patientid|offset|sex|age| finding|view|modality| location|sex_cat|finding_cat|view_cat|modality_cat|location_cat|
+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+
 2| 0| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|
 2| 3| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|
 2| 5| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|
 2| 6| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|
 4| 0| F| 52|COVID-19| PA| X-ray|Changhua Christia...| 1.0| 0.0| 0.0| 0.0| 20.0|
+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+
only showing top 5 rows

**1.3 Creating Features Column**


In [ ]:
assembler = VectorAssembler(inputCols=['offset','age','sex_cat','finding_cat','view_cat','modality_cat','location_cat'],
                           outputCol='features') 
#so we're grabbing columns 1 to 2nd last one which are our features. Last column is our label. Column 0 is ID which is not relevant

output = assembler.transform(encoded) #applying our vector assembler to all our data



Out[13]: [Row(features=SparseVector(7, {1: 65.0, 6: 11.0}))]

In [ ]:
output.select('features').head(4) #notice features is a DenseVector containing all the features we combined

Out[16]: [Row(features=SparseVector(7, {1: 65.0, 6: 11.0})),
 Row(features=SparseVector(7, {0: 3.0, 1: 65.0, 6: 11.0})),
 Row(features=SparseVector(7, {0: 5.0, 1: 65.0, 6: 11.0})),
 Row(features=SparseVector(7, {0: 6.0, 1: 65.0, 6: 11.0}))]

**1.4 Scaling Features**


In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

scalerModel = scaler.fit(output)

# Normalize each feature to have unit standard deviation.
cluster_final = scalerModel.transform(output)
cluster_final.show(5)

+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+--------------------+--------------------+
patientid|offset|sex|age| finding|view|modality| location|sex_cat|finding_cat|view_cat|modality_cat|location_cat| features| scaledFeatures|
+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+--------------------+--------------------+
 2| 0| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|(7,[1,6],[65.0,11...|(7,[1,6],[4.30266...|
 2| 3| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|(7,[0,1,6],[3.0,6...|(7,[0,1,6],[0.419...|
 2| 5| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|(7,[0,1,6],[5.0,6...|(7,[0,1,6],[0.698...|
 2| 6| M| 65|COVID-19| PA| X-ray|Cho Ray Hospital,...| 0.0| 0.0| 0.0| 0.0| 11.0|(7,[0,1,6],[6.0,6...|(7,[0,1,6],[0.838...|
 4| 0| F| 52|COVID-19| PA| X-ray|Changhua Christia...| 1.0| 0.0| 0.0| 0.0| 20.0|(7,[1,2,6],[52.0,...|(7,[1,2,6],[3.442...|
+---------+------+---+---+--------+----+--------+--------------------+-------+-----------+--------+------------+------------+--------------------+--------------------+
only showing top 5 rows

#**2.0 Creating Cluster**


In [ ]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

model_k3 = kmeans3.fit(cluster_final)
model_k2 = kmeans2.fit(cluster_final)

pred_k3 = model_k3.transform(cluster_final)
pred_k2 = model_k2.transform(cluster_final)

evaluator = ClusteringEvaluator()

silhouette_k3 = evaluator.evaluate(pred_k3)
silhouette_k2 = evaluator.evaluate(pred_k2)

print("Silhouette with squared euclidean distance = " + str(silhouette_k3))
print("Silhouette with squared euclidean distance = " + str(silhouette_k2))

Silhouette with squared euclidean distance = -0.10475149410141821
Silhouette with squared euclidean distance = -0.0387862596451575

In [ ]:
centers_k3 = model_k3.clusterCenters()
print("Cluster Centers: ")
for center in centers_k3:
    print(center)

Cluster Centers: 
[0.91758363 3.77801394 0. 0.16447591 0.54902394 0.
 0.63020525]
[1.25299969 3.58693739 0.62818219 0.1513926 2.61730008 1.77403195
 0.5752696 ]
[0.865409 3.62601078 1.9822638 0.7064988 0.26764917 0.
 1.12635368]

In [ ]:
centers_k2 = model_k2.clusterCenters()
print("Cluster Centers: ")
for center in centers_k2:
    print(center)

Cluster Centers: 
[0.81675027 3.699787 1.09786918 0.49377278 0.2337575 0.
 0.95908564]
[1.28276687 3.6527562 0.36548782 0.08808297 2.12921467 1.03216404
 0.47039244]